In [13]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Define the paths to your dataset folders
train_dataset_dir = r"D:\datasets\Underwater_Image\LIMUC (Labeled Images for Ulcerative Colitis)\train_and_validation_sets\train_and_validation_sets"
test_dataset_dir = r"D:\datasets\Underwater_Image\LIMUC (Labeled Images for Ulcerative Colitis)\test_set\test_set"

# Initialize empty lists for X_train, Y_train, X_test, and Y_test
X_train = []
Y_train = []
X_test = []
Y_test = []

# Initialize an empty list to store categorical labels
categorical_labels = []

# Define a function to read and preprocess images
def process_images(folder_path, label, is_train_set=True):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and filename.endswith(".bmp"):  # Check if it's a file and ends with .bmp
            # Open and resize the image to (32, 32, 3)
            img = Image.open(file_path)
            img = img.resize((32, 32))
            img = img.convert("RGB")
            
            # Convert image data to a NumPy array
            img_array = np.array(img).astype('float32')  # Convert to float
            
            # Normalize the image data (optional)
            img_array /= 255.0  # Normalize pixel values to [0, 1]
            
            # Append the image data to the appropriate list
            if is_train_set:
                X_train.append(img_array)
                Y_train.append(label)  # Append the numerical label
            else:
                X_test.append(img_array)
                Y_test.append(label)  # Append the numerical label
            
            # Append the label for categorical encoding
            categorical_labels.append(label)  # Append the numerical label

# List the folders inside the training dataset directory
train_folders = os.listdir(train_dataset_dir)

# Create a label encoder for categorical labels
label_encoder = LabelEncoder()

# Loop through the training folders and process images
for label, folder_name in enumerate(train_folders):
    folder_path = os.path.join(train_dataset_dir, folder_name)
    if os.path.isdir(folder_path):  # Check if it's a directory
        process_images(folder_path, label)

# List the folders inside the test dataset directory
test_folders = os.listdir(test_dataset_dir)

# Loop through the test folders and process images
for label, folder_name in enumerate(test_folders):
    folder_path = os.path.join(test_dataset_dir, folder_name)
    if os.path.isdir(folder_path):  # Check if it's a directory
        process_images(folder_path, label, is_train_set=False)
        
        
        

# Encode Y_train and Y_test categorically
num_classes = len(np.unique(categorical_labels))
  # Update this to match the number of classes

# Convert Y_train and Y_test to NumPy arrays
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

# Make sure your labels are integers ranging from 0 to num_classes - 1
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

# One-hot encode the labels
Y_train_categorical = to_categorical(Y_train, num_classes=num_classes)
Y_test_categorical = to_categorical(Y_test, num_classes=num_classes)



# # Encode Y_train and Y_test categorically using the label encoder
# num_classes = len(np.unique(Y_train))  # Automatically determine the number of classes
# Y_train_categorical = to_categorical(Y_train, num_classes=num_classes)
# Y_test_categorical = to_categorical(Y_test, num_classes=num_classes)


# Check the shape of X_train, Y_train_categorical, X_test, and Y_test_categorical
print("Shape of X_train:", np.shape(X_train))
print("Shape of Y_train_categorical:", np.shape(Y_train_categorical))
print("Shape of X_test:", np.shape(X_test))
print("Shape of Y_test_categorical:", np.shape(Y_test_categorical))

Shape of X_train: (9590, 32, 32, 3)
Shape of Y_train_categorical: (9590, 4)
Shape of X_test: (1686, 32, 32, 3)
Shape of Y_test_categorical: (1686, 4)


In [10]:
train_folders

['Mayo 0', 'Mayo 1', 'Mayo 2', 'Mayo 3']

In [14]:
from tensorflow.keras.applications.mobilenet import MobileNet
# Create the ResNet model without the top (fully connected) layers
#base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model = MobileNet(weights="imagenet", include_top=False, input_shape=(32, 32, 3))
# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))  # Change units to match the number of classes
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 1, 1, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d_4   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_4 (Dense)             (None, 4)                 4100      
                                                                 
Total params: 3,232,964
Trainable params: 3,211,076
Non-trainable params: 21,888
_________________________________________________________________


In [15]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(np.array(X_train).astype('float32'), Y_train_categorical, batch_size=64, epochs=100, validation_split = .2)

Epoch 1/100
120/120 [==============================] - 7s 29ms/step - loss: 0.9070 - accuracy: 0.6794 - val_loss: 31.6965 - val_accuracy: 5.2138e-04
Epoch 2/100
120/120 [==============================] - 3s 23ms/step - loss: 0.5385 - accuracy: 0.7466 - val_loss: 20.5966 - val_accuracy: 0.0010
Epoch 3/100
120/120 [==============================] - 2s 20ms/step - loss: 0.5013 - accuracy: 0.7663 - val_loss: 11.8314 - val_accuracy: 5.2138e-04
Epoch 4/100
120/120 [==============================] - 2s 20ms/step - loss: 0.4813 - accuracy: 0.7763 - val_loss: 13.0816 - val_accuracy: 0.0162
Epoch 5/100
120/120 [==============================] - 2s 18ms/step - loss: 0.4621 - accuracy: 0.7871 - val_loss: 13.4982 - val_accuracy: 0.0193
Epoch 6/100
120/120 [==============================] - 3s 22ms/step - loss: 0.4311 - accuracy: 0.8012 - val_loss: 12.1383 - val_accuracy: 0.0334
Epoch 7/100
120/120 [==============================] - 3s 21ms/step - loss: 0.4227 - accuracy: 0.8127 - val_loss: 11.8250 

120/120 [==============================] - 3s 21ms/step - loss: 0.0374 - accuracy: 0.9881 - val_loss: 14.5873 - val_accuracy: 0.0193
Epoch 58/100
120/120 [==============================] - 2s 20ms/step - loss: 0.0398 - accuracy: 0.9851 - val_loss: 18.1657 - val_accuracy: 0.0328
Epoch 59/100
120/120 [==============================] - 3s 21ms/step - loss: 0.0365 - accuracy: 0.9875 - val_loss: 17.7715 - val_accuracy: 0.0261
Epoch 60/100
120/120 [==============================] - 3s 24ms/step - loss: 0.0404 - accuracy: 0.9853 - val_loss: 17.1806 - val_accuracy: 0.0167
Epoch 61/100
120/120 [==============================] - 4s 35ms/step - loss: 0.0329 - accuracy: 0.9872 - val_loss: 20.6729 - val_accuracy: 0.0250
Epoch 62/100
120/120 [==============================] - 4s 32ms/step - loss: 0.0485 - accuracy: 0.9847 - val_loss: 15.9419 - val_accuracy: 0.0276
Epoch 63/100
120/120 [==============================] - 5s 44ms/step - loss: 0.0314 - accuracy: 0.9901 - val_loss: 15.7775 - val_accuracy

In [16]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, balanced_accuracy_score
batch_size = 64
# Evaluate the model on the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test_categorical))
test_dataset = test_dataset.batch(batch_size)

test_loss, test_accuracy = model.evaluate(test_dataset)

# Make predictions on the test dataset
predictions = model.predict(test_dataset)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

# Calculate balanced accuracy
balanced_acc = balanced_accuracy_score(Y_test, predicted_labels)

# Calculate confusion matrix
conf_matrix = confusion_matrix(Y_test, predicted_labels)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Balanced Accuracy: {balanced_acc}")
print("Cohen Kappa Score:")
from sklearn.metrics import cohen_kappa_score
k = cohen_kappa_score(Y_test, predicted_labels, weights='quadratic')
print(k)

27/27 [==============================] - 1s 6ms/step
Test Loss: 5.340921878814697
Test Accuracy: 0.6043890714645386
Accuracy: 0.6043890865954923
F1 Score: 0.5437447255985512
Balanced Accuracy: 0.3379712255358807
Cohen Kappa Score:
0.2352167374411619


In [17]:
from tensorflow import keras
model = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\limuc_mob_32_model.h5")

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D()(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the new model with the updated head
new_model = keras.models.Model(inputs=model.input, outputs=output)
# Compile the model
new_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
new_model.fit(np.array(X_train).astype('float32'), Y_train_categorical, batch_size=64, epochs=100, validation_split = .2)

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
120/120 [==============================] - 7s 28ms/step - loss: 0.6653 - accuracy: 0.7061 - val_loss: 16.7211 - val_accuracy: 0.0302
Epoch 2/100
120/120 [==============================] - 3s 24ms/step - loss: 0.4973 - accuracy: 0.7598 - val_loss: 15.1286 - val_accuracy: 0.0370
Epoch 3/100
120/120 [==============================] - 4s 31ms/step - loss: 0.4721 - accuracy: 0.7703 - val_loss: 14.2042 - val_accuracy: 0.0193
Epoch 4/100
120/120 [==============================] - 4s 30ms/step - loss: 0.4455 - accuracy: 0.7804 - val_loss: 14.6593 - val_accuracy: 0.0235
Epoch 5/100
120/120 [==============================] - 3s 26ms/step - loss: 0.4228 - accuracy: 0.8015 - val_loss: 13.2441 - val_accuracy: 0.0308
Epoch 6/100
120/120 [==============================] - 3s 22ms/step - loss: 0.4153 - accuracy: 0.8070 - val_loss: 16.0926 - val_accuracy: 0.0219
Epoch 7/100
120/120 [==============================] - 3s 21ms/step - loss: 0.3846 - accuracy: 0.8187 - val_loss: 17.9151 - val_ac

120/120 [==============================] - 3s 23ms/step - loss: 0.0280 - accuracy: 0.9917 - val_loss: 16.2830 - val_accuracy: 0.0209
Epoch 58/100
120/120 [==============================] - 3s 21ms/step - loss: 0.0314 - accuracy: 0.9896 - val_loss: 16.7652 - val_accuracy: 0.0276
Epoch 59/100
120/120 [==============================] - 3s 21ms/step - loss: 0.0228 - accuracy: 0.9923 - val_loss: 18.6482 - val_accuracy: 0.0240
Epoch 60/100
120/120 [==============================] - 3s 21ms/step - loss: 0.0270 - accuracy: 0.9906 - val_loss: 19.0058 - val_accuracy: 0.0240
Epoch 61/100
120/120 [==============================] - 2s 19ms/step - loss: 0.0307 - accuracy: 0.9894 - val_loss: 20.5117 - val_accuracy: 0.0261
Epoch 62/100
120/120 [==============================] - 2s 20ms/step - loss: 0.0221 - accuracy: 0.9920 - val_loss: 18.5418 - val_accuracy: 0.0250
Epoch 63/100
120/120 [==============================] - 2s 20ms/step - loss: 0.0443 - accuracy: 0.9850 - val_loss: 18.4401 - val_accuracy

In [18]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, balanced_accuracy_score
batch_size = 64
# Evaluate the model on the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test_categorical))
test_dataset = test_dataset.batch(batch_size)

test_loss, test_accuracy = new_model.evaluate(test_dataset)

# Make predictions on the test dataset
predictions = new_model.predict(test_dataset)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

# Calculate balanced accuracy
balanced_acc = balanced_accuracy_score(Y_test, predicted_labels)

# Calculate confusion matrix
conf_matrix = confusion_matrix(Y_test, predicted_labels)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Balanced Accuracy: {balanced_acc}")
print("Cohen Kappa Score:")
from sklearn.metrics import cohen_kappa_score
k = cohen_kappa_score(Y_test, predicted_labels, weights='quadratic')
print(k)

27/27 [==============================] - 1s 18ms/step
Test Loss: 3.823970079421997
Test Accuracy: 0.599644124507904
Accuracy: 0.599644128113879
F1 Score: 0.5201740133923423
Balanced Accuracy: 0.3148642823858341
Cohen Kappa Score:
0.19699459057737745
